In [9]:
from math import log 
from collections import defaultdict
from tqdm import tqdm

frequencyFileList = ["5000frequency.txt", "10000frequency.txt", "15000frequency.txt", "20000frequency.txt", "25000frequency.txt", "30000frequency.txt", "35000frequency.txt", "40000frequency.txt", "45000frequency.txt"]
freqdict = defaultdict(int)
possibleValues = []

for fileName in tqdm(frequencyFileList):
    file = open("frequencyLists/" + fileName)
    allString = file.read()
    for index, word in enumerate(allString.split(" ")[1::2]):
        possibleValues.append(int(allString.split(" ")[index*2]))
        freqdict[word] = log(int(allString.split(" ")[index*2]))

100%|██████████| 9/9 [00:49<00:00,  5.54s/it]


In [10]:
from spellchecker import SpellChecker

spell = SpellChecker(language='de')

def lookup(string, spell=spell):
    try:
        return freqdict[string]
    except:
        try:
            return freqdict[spell.correction(string)]
        except:
            return 0

In [11]:
import pandas as pd
df=pd.read_csv(r"preprocessed_text_df.csv", sep="|", lineterminator='\n')
import ast
display(df.head())
df["preprocessedText"]  = [ast.literal_eval(text) for text in df["RFTagger"]]

def save_df(path,df):
    df.to_csv(path,sep="|",index=False)

,Level,Title,Text,Source,Type,cleanedText,RFTagger,parsedText,newLevel
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading,Zu meiner Familie gehören vier Personen. Die M...,"[[['Zu', 'APPR', 'Dat'], ['meiner', 'PRO', 'Po...",['(ROOT (S (PP (APPR Zu) (PPOSAT meiner) (NN F...,A
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,"[[['Mein', 'PRO', 'Poss', 'Attr', '-', 'Nom', ...",['(ROOT (S (NP (PPOSAT Mein) (NN Name)) (VAFIN...,A
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,"[[['Hallo', 'ITJ'], ['!', 'SYM', 'Pun', 'Sent'...","['(ROOT (NUR (ITJ Hallo) ($. !)))', '(ROOT (S ...",A
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,"[[['Ricarda', 'N', 'Name', 'Nom', 'Sg', 'Fem']...",['(ROOT (CS (S (NE Ricarda) (VAFIN ist) (AP (N...,A
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading,Frau Meier geht heute in den Supermarkt. Ihr M...,"[[['Frau', 'N', 'Reg', 'Nom', 'Sg', 'Fem'], ['...",['(ROOT (S (NP (NN Frau) (NE Meier)) (VVFIN ge...,A


In [12]:
import numpy as np


meanFreq = []
medianFreq = []
stdFreq = []
bin0=[]
bin1=[]
bin2=[]
bin3=[]
bin4=[]
bin5=[]
bin6=[]
bin7=[]
bin8=[]
bin9=[]
bin10=[]
bin11=[]
bin12=[]
bin13=[]
bin14=[]

for text in tqdm(df["preprocessedText"]):
    freqvalues=[]
    
    for sentence in text:
        for word in sentence:
            if word[1]!= "SYM":
                freqvalues.append(lookup(word[0]))
    meanFreq.append(np.mean(freqvalues))
    medianFreq.append(np.median(freqvalues))
    stdFreq.append(np.std(freqvalues))
    bin0.append(sum([1 for freqvalue in freqvalues if freqvalue == 0 ])/len(freqvalues))
    bin1.append(sum([1 for freqvalue in freqvalues if freqvalue > 0 and freqvalue < 1 ])/len(freqvalues))
    bin2.append(sum([1 for freqvalue in freqvalues if freqvalue > 1 and freqvalue < 2 ])/len(freqvalues))
    bin3.append(sum([1 for freqvalue in freqvalues if freqvalue > 2 and freqvalue < 3 ])/len(freqvalues))
    bin4.append(sum([1 for freqvalue in freqvalues if freqvalue > 3 and freqvalue < 4 ])/len(freqvalues))
    bin5.append(sum([1 for freqvalue in freqvalues if freqvalue > 4 and freqvalue < 5 ])/len(freqvalues))
    bin6.append(sum([1 for freqvalue in freqvalues if freqvalue > 5 and freqvalue < 6 ])/len(freqvalues))
    bin7.append(sum([1 for freqvalue in freqvalues if freqvalue > 6 and freqvalue < 7 ])/len(freqvalues))
    bin8.append(sum([1 for freqvalue in freqvalues if freqvalue > 7 and freqvalue < 8 ])/len(freqvalues))
    bin9.append(sum([1 for freqvalue in freqvalues if freqvalue > 8 and freqvalue < 9 ])/len(freqvalues))
    bin10.append(sum([1 for freqvalue in freqvalues if freqvalue > 9 and freqvalue < 10 ])/len(freqvalues))
    bin11.append(sum([1 for freqvalue in freqvalues if freqvalue > 10 and freqvalue < 11 ])/len(freqvalues))
    bin12.append(sum([1 for freqvalue in freqvalues if freqvalue > 11 and freqvalue < 12 ])/len(freqvalues))
    bin13.append(sum([1 for freqvalue in freqvalues if freqvalue > 12 and freqvalue < 13 ])/len(freqvalues))
    bin14.append(sum([1 for freqvalue in freqvalues if freqvalue > 13 and freqvalue < 14 ])/len(freqvalues))
    
    
df["FF-meanFreq"] = meanFreq
df["FF-medianFreq"] = medianFreq
df["FF-bin0"] = bin0
df["FF-bin1"] = bin1
df["FF-bin2"] = bin2
df["FF-bin3"] = bin3
df["FF-bin4"] = bin4
df["FF-bin5"] = bin5
df["FF-bin6"] = bin6
df["FF-bin7"] = bin7
df["FF-bin8"] = bin8
df["FF-bin9"] = bin9
df["FF-bin10"] = bin10
df["FF-bin11"] = bin11
df["FF-bin12"] = bin12
df["FF-bin13"] = bin13
df["FF-bin14"] = bin14

display(df.head())
filename = "frequency_features_df.csv"
save_df(filename,df)

100%|██████████| 1431/1431 [00:00<00:00, 2388.52it/s]


,Level,Title,Text,Source,Type,cleanedText,RFTagger,parsedText,newLevel,preprocessedText,...,FF-bin5,FF-bin6,FF-bin7,FF-bin8,FF-bin9,FF-bin10,FF-bin11,FF-bin12,FF-bin13,FF-bin14
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading,Zu meiner Familie gehören vier Personen. Die M...,"[[['Zu', 'APPR', 'Dat'], ['meiner', 'PRO', 'Po...",['(ROOT (S (PP (APPR Zu) (PPOSAT meiner) (NN F...,A,"[[[Zu, APPR, Dat], [meiner, PRO, Poss, Attr, -...",...,0.019231,0.000000,0.028846,0.105769,0.163462,0.163462,0.115385,0.134615,0.201923,0.009615
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,"[[['Mein', 'PRO', 'Poss', 'Attr', '-', 'Nom', ...",['(ROOT (S (NP (PPOSAT Mein) (NN Name)) (VAFIN...,A,"[[[Mein, PRO, Poss, Attr, -, Nom, Sg, Masc], [...",...,0.033113,0.039735,0.072848,0.105960,0.152318,0.132450,0.059603,0.145695,0.139073,0.052980
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,"[[['Hallo', 'ITJ'], ['!', 'SYM', 'Pun', 'Sent'...","['(ROOT (NUR (ITJ Hallo) ($. !)))', '(ROOT (S ...",A,"[[[Hallo, ITJ], [!, SYM, Pun, Sent]], [[Ich, P...",...,0.009346,0.056075,0.037383,0.046729,0.084112,0.158879,0.056075,0.130841,0.140187,0.046729
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,"[[['Ricarda', 'N', 'Name', 'Nom', 'Sg', 'Fem']...",['(ROOT (CS (S (NE Ricarda) (VAFIN ist) (AP (N...,A,"[[[Ricarda, N, Name, Nom, Sg, Fem], [ist, VFIN...",...,0.020000,0.060000,0.110000,0.060000,0.150000,0.080000,0.100000,0.090000,0.140000,0.030000
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading,Frau Meier geht heute in den Supermarkt. Ihr M...,"[[['Frau', 'N', 'Reg', 'Nom', 'Sg', 'Fem'], ['...",['(ROOT (S (NP (NN Frau) (NE Meier)) (VVFIN ge...,A,"[[[Frau, N, Reg, Nom, Sg, Fem], [Meier, N, Nam...",...,0.105727,0.044053,0.074890,0.066079,0.070485,0.171806,0.083700,0.101322,0.145374,0.026432
